In [97]:
import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx
import random
from joblib import parallel, delayed

Spectral Clustering using Adjacency Matrix and Laplacian:
Adjacency:  optimizes on in group closeness
Laplacian: Average cut
A variant of the Laplacian: Normalized Cut

In general, Laplacian has better behaved/more stable convergenence properties of eigenvectors (makes sense when we are dealing with random graphs).  

TODO
make function for laplacian
make function for adjaceny matrix
make function for non backtracking matrix

make sampling method for blockmodel

make function for power method

make function for power method for subspace

In [14]:
#assume graphs are dictionaries of vertices with entries that are the incident vertices for now, we can change this later.  



#compute laplacian
    #f-Mf

        #averaging function
        
simple_graph = {
    0: [2,3],
    1: [2, 3],
    2: [3], 
    3: [0, 1]
}

simple_graph[1]

vector = np.array([1,1,1,1])

lst = [vector[i] for i in simple_graph[1]]


In [15]:
def AverageOperatorVertex(vertex, vector, graph):
    lst = [vector[i] for i in graph[vertex]]
    answer = np.sum(lst)
    return(answer)



In [16]:
AverageOperatorVertex(vertex = 2, vector = np.array([1,2,3,4]), graph = simple_graph)

4

In [24]:

def GraphLaplacian(graph, vector):
    #takes in graph and vector on graph indices, returns new vector
    new_vector = np.empty(len(vector),)
    for i in graph:
        #this step can be parallelized
        new_vector[i] = vector[i]*len(graph[i])-AverageOperatorVertex(vertex=i, vector = vector, graph = graph)
    return new_vector

In [38]:
er_graph_10 = nx.erdos_renyi_graph(10, 0.2, seed=1)
nx.to_dict_of_lists(er_graph_10)
print sp.sparse.csgraph.laplacian(nx.to_numpy_matrix(er_graph_10))
print np.dot(sp.sparse.csgraph.laplacian(nx.to_numpy_matrix(er_graph_10)), [1,0,1,0,1,0,1,0,1,0])

GraphLaplacian(nx.to_dict_of_lists(er_graph_10), [1,0,1,0,1,0,1,0,1,0])

#yes!  They did the same thing!!!! So correct function confirmed 

[[ 2. -1. -0. -0. -0. -0. -0. -0. -0. -1.]
 [-1.  3. -1. -0. -0. -0. -1. -0. -0. -0.]
 [-0. -1.  3. -0. -0. -1. -1. -0. -0. -0.]
 [-0. -0. -0.  1. -0. -0. -1. -0. -0. -0.]
 [-0. -0. -0. -0.  0. -0. -0. -0. -0. -0.]
 [-0. -0. -1. -0. -0.  2. -1. -0. -0. -0.]
 [-0. -1. -1. -1. -0. -1.  5. -1. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -1.  2. -1. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -1.  1. -0.]
 [-1. -0. -0. -0. -0. -0. -0. -0. -0.  1.]]
[ 2. -3.  2. -1.  0. -2.  4. -2.  1. -1.]


array([ 2., -3.,  2., -1.,  0., -2.,  4., -2.,  1., -1.])

In [148]:
#this is problematic for the spectral algorithm
#as the unnormalized laplacian requires extraction of k smallest eigenvector, so far
#we are extracting the k largest eigenvectors


#first try at implementing net

#nonlinearity?
#fixed number of layers for iterations
#fixed subspace dimension for number of eigenvectors


def LaplacianEigenvalueExtract(Graph, layers=10, subspace_dim=1, vector=None):
    #takes in graph, computes Laplacian, then does powermethod
    #we end up with a list of k eigenvectors to do k-means on
    #we assume Graph given as a dictionary
    
    if vector==None:
        vector = np.random.normal(0,1,len(Graph)) #we initalize with function with random components as default
    
    eigenvectors = [vector]*subspace_dim #initialize list of eigvectors we want to return
    
    new_vector = vector 
    lmbda_lst = np.zeros(subspace_dim)#ghetto hack for operator tail recursion
    for i in range(subspace_dim): #This is done for the number of eigenvectors we want to find.
        #so this cannot be done in parallel.  

        for j in range(layers): #This is iterated for the power we want to bring the operator, 
            #so this cannot be done in parallel:
            tmp_vector = GraphLaplacian(graph=Graph, vector=new_vector)
            for k in range(subspace_dim):
                tmp_vector = tmp_vector-lmbda_lst[k]*new_vector 
                
                #this will only amount to something if our list contains eigenvalues
            new_vector = tmp_vector/np.linalg.norm(tmp_vector)

           
        eigenvectors[i] = new_vector #this will already be normalized by the last iteration of j
        lmbda_lst[i] = np.dot(GraphLaplacian(graph=Graph, vector = eigenvectors[i]), eigenvectors[i]) #since it is normalized,
        #the scaling factor will be lamba
        print lmbda_lst[i]

        new_vector = np.random.normal(0,1,len(Graph))
        #now we are onto finding the second eigenvector
        #we should randomize the next vector
    

        
    return eigenvectors
        

In [ ]:
random.seed(2)

LaplacianEigenvalueExtract(Graph = nx.to_dict_of_lists(er_graph_10), layers =100000, subspace_dim=2, 
                          vector = np.random.normal(0,1,10))


/Users/lishali/.virtualenvs/stitchfix/lib/python2.7/site-packages/ipykernel/__main__.py:18: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [150]:
#does seem to be very close....


#omg that worked, this is just coded insanely terribly, read the last eigenvector 
np.linalg.eigh(sp.sparse.csgraph.laplacian(nx.to_numpy_matrix(er_graph_10)))

(array([ -2.26725847e-16,   2.26725847e-16,   2.99157099e-01,
          5.50092286e-01,   1.05880186e+00,   2.00000000e+00,
          2.47466903e+00,   3.10172249e+00,   4.40522337e+00,
          6.11033387e+00]),
 array([[  3.31608274e-01,  -3.38683298e-02,  -4.34881048e-01,
          -1.61958943e-01,  -1.15691988e-02,   5.00000000e-01,
          -9.59407281e-02,   5.71772005e-01,   2.87798428e-01,
          -6.66860830e-02],
        [  3.31608274e-01,  -3.38683298e-02,  -1.19152888e-01,
           1.25157067e-01,  -2.07637763e-01,   5.00000000e-01,
          -1.95190675e-02,  -3.57884834e-01,  -6.07702749e-01,
           2.61052804e-01],
        [  3.31608274e-01,  -3.38683298e-02,   1.63185651e-02,
           2.65781018e-01,  -3.45128199e-01,   2.22044605e-16,
          -7.49681661e-02,  -4.53440598e-01,   6.83402450e-01,
           1.43534176e-01],
        [  3.31608274e-01,  -3.38683298e-02,   1.38046986e-01,
           4.50761749e-01,   7.88557356e-01,  -4.44089210e-16,
         

In [144]:


#first try at implementing net for adjacnecy matrices

def AdjacencyEigenvalueExtract(Graph, layers=10, subspace_dim=1, vector=None):
    #takes in graph, computes Laplacian, then does powermethod
    #we end up with a list of k eigenvectors to do k-means on
    #we assume Graph given as a dictionary
        
        
    if vector==None:
        vector = np.random.normal(0,1,len(Graph)) #we initalize with function with random components as default
    
    eigenvectors = [vector]*subspace_dim #initialize list of eigvectors we want to return
         
    new_matrix = Graph
    
    for i in range(subspace_dim): #This is done for the number of eigenvectors we want to find.
        #so this cannot be done in parallel.  

        for j in range(layers): #This is iterated for the power we want to bring the operator, 
            #so this cannot be done in parallel:
            tmp_vector = np.multiply(new_matrix, vector) #this should be done in parallel since each component does not reference the other

            new_vector = tmp_vector/np.linalg.norm(tmp_vector)
            
        eigenvectors[i] = new_vector #this will already be normalized by the last iteration of j
        

        new_matrix = new_matrix - np.dot(new_matrix, eigenvectors[i])*np.identity(len(Graph))
        
        vector = np.random.normal(0,1,len(Graph))
        #now we are onto finding the second eigenvector
        #we should randomize the next vector
    

        
    return eigenvectors
        

In [145]:
er_graph_10 = nx.erdos_renyi_graph(10, 0.2, seed=1)

AdjacencyEigenvalueExtract(Graph = nx.to_numpy_matrix(er_graph_10),
                           layers = 100, 
                           subspace_dim=4)

np.linalg.eigh(nx.to_numpy_matrix(er_graph_10))

[matrix([[-0.        ,  0.11789212, -0.        , -0.        ,  0.        ,
          -0.        ,  0.        ,  0.        , -0.        , -0.04784204],
         [-0.12708011,  0.        , -0.14307883, -0.        ,  0.        ,
          -0.        ,  0.37871455,  0.        , -0.        , -0.        ],
         [-0.        ,  0.11789212, -0.        , -0.        ,  0.        ,
          -0.12897782,  0.37871455,  0.        , -0.        , -0.        ],
         [-0.        ,  0.        , -0.        , -0.        ,  0.        ,
          -0.        ,  0.37871455,  0.        , -0.        , -0.        ],
         [-0.        ,  0.        , -0.        , -0.        ,  0.        ,
          -0.        ,  0.        ,  0.        , -0.        , -0.        ],
         [-0.        ,  0.        , -0.14307883, -0.        ,  0.        ,
          -0.        ,  0.37871455,  0.        , -0.        , -0.        ],
         [-0.        ,  0.11789212, -0.14307883, -0.03651975,  0.        ,
          -0.128977

In [146]:
np.linalg.eigh(nx.to_numpy_matrix(er_graph_10))

(array([ -2.16045879e+00,  -1.59820643e+00,  -1.00000000e+00,
         -7.54713673e-01,  -2.47509086e-18,   1.06992180e-16,
          3.02324782e-01,   1.00000000e+00,   1.31474495e+00,
          2.89630916e+00]),
 matrix([[ -2.54774785e-01,   4.22714525e-01,  -5.00000000e-01,
            5.10966382e-02,   9.81277765e-18,  -2.60778306e-17,
            3.69437993e-02,   2.88675135e-01,   6.26003278e-01,
            1.64963909e-01],
         [  4.32504183e-01,  -4.11092000e-01,  -3.33066907e-16,
            2.91400108e-02,   5.50611531e-01,   1.73667140e-01,
           -1.11030018e-01,   1.66533454e-16,   3.46894175e-01,
            4.20829885e-01],
         [ -5.81230860e-02,   5.40608665e-01,   5.00000000e-01,
            2.15643334e-01,  -2.51277390e-17,   1.47073421e-16,
           -3.18484306e-01,  -2.88675135e-01,   2.06165114e-02,
            4.71880603e-01],
         [  2.87674819e-01,   1.91660669e-01,  -2.22044605e-16,
            3.82572023e-01,   1.84358899e-16,  -4.62838375e

In [46]:
er_graph_10 = nx.erdos_renyi_graph(10, 0.2, seed=1)
print nx.to_dict_of_lists(er_graph_10)
print nx.to_numpy_matrix(er_graph_10)

{0: [1, 9], 1: [0, 2, 6], 2: [1, 5, 6], 3: [6], 4: [], 5: [2, 6], 6: [1, 2, 3, 5, 7], 7: [8, 6], 8: [7], 9: [0]}
[[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  1.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  1.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  1.  1.  1.  0.  1.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [47]:
len(nx.to_numpy_matrix(er_graph_10))

10